In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from nltk import ngrams
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from  sklearn import metrics

import numpy as np
import textblob
import re, nltk
from nltk.stem import WordNetLemmatizer
import nltk
import collections
from nltk.corpus import stopwords
#directly downloads packages for required things
nltk.download('punkt')
nltk.download('popular')
from nltk.corpus import stopwords
import  pandas as pd
import string
#importing the data
from collections import Counter
#pass the test dataset here
test_data=pd.read_csv(r'data2.csv',engine='python')
print("given size of the data",len(test_data))
#dropping the null values
test_data=test_data.drop_duplicates()
print(" size of data after removing repitations",len(test_data))
#print(test_data['TWEETS'].value_counts())
#DATA PRECPROCESSING STARTS FROM HERRE

wordnet_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def normalizer(tweet):
    aplhabets = re.sub("[^a-zA-Z]", " ",tweet)
    tokens = nltk.word_tokenize(aplhabets)[2:]
    lower_case = [l.lower() for l in tokens]
    processed_words = list(filter(lambda l: l not in stop_words, lower_case))
    lemmas = [wordnet_lemmatizer.lemmatize(t) for t in processed_words]
    return lemmas

#create a coloum in the dataset to get only normalized words
print("start of pre_processing")
test_data["Review Text"]=test_data["Review Text"].astype(str)
test_data["normalized_Tweets"]=np.nan
test_data["normalized_Tweets"]=test_data["Review Text"].apply(lambda x: normalizer(x))
#now we are converting to ngrams to get the meanining

test_data["processed_tweets"]=np.nan
test_data["sentiment"]=np.nan
def sentiment(tweet):
    return textblob.TextBlob(tweet).sentiment.polarity
def feauture_labelling(score):
    if (score >= 1):
        return 5

    elif (score <= -1):
        return 1

    elif ((score > -0.5) and (score < 0.5)):
        return 3

    elif ((score >= 0.5) and (score < 1)):
        return 4

    elif ((score <= -0.5) and (score > -1)):
        return 2
#note 1=en,2=n,3=nue,4=p,5=exp


test_data["processed_tweets"]=test_data["normalized_Tweets"].apply(lambda x: " ".join(list(x)))
#print(test_data["processed_tweets"].head())

test_data["sentiment"]=test_data["processed_tweets"].apply(lambda x: sentiment(x))
#print(test_data["sentiment"].value_counts())
test_data["Target_label"]=np.nan
test_data["Target_label"]=test_data["sentiment"].apply(lambda x: feauture_labelling(x))
print("displaying the count of data based on caluculated rating")
print(test_data["Target_label"].value_counts())
test_data.to_csv('.\preprocessed_reviews2.csv')

from sklearn.model_selection import train_test_split

final_df=test_data[['sentiment','Target_label']]
#print(test_data['sentiment'].value_counts())
#print(test_data['Target_label'].value_counts())
train , test = train_test_split(final_df, test_size = 0.15)
x_train = train.drop('Target_label', axis=1)

y_train = train['Target_label']

x_test = test.drop('Target_label', axis = 1)
y_test = test['Target_label']


clf1 =  RandomForestClassifier(n_estimators=5000,max_depth=1)


#coding testing and saving with svc    
#Train the model using the training sets
clf1.fit(x_train, y_train)
#Predict the response for test dataset
y_pred1 = clf1.predict(x_test)
#print(y_pred)
#saving predictions


from sklearn.metrics import f1_score
print("ramdom forest classcification model accuracy(in %):",metrics.accuracy_score(y_test, y_pred1) * 100)





[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Krishna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\Krishna\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\Krishna\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\Krishna\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\Krishna\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\K

In [14]:
from sklearn.model_selection import train_test_split

final_df=test_data[['sentiment','Target_label']]
print(test_data['sentiment'].value_counts())
print(test_data['Target_label'].value_counts())
train , test = train_test_split(final_df, test_size = 0.3)
x_train = train.drop('Target_label', axis=1)

y_train = train['Target_label']

x_test = test.drop('Target_label', axis = 1)
y_test = test['Target_label']
clf2 = XGBClassifier()




#Train the model using the training sets
clf2.fit(x_train, y_train)
#Predict the response for test dataset
y_pred2 = clf2.predict(x_test)
#print(y_pred)
#saving predictions

print("xgboost classcification model accuracy(in %):", metrics.accuracy_score(y_test, y_pred2) * 100)






 0.000000    5934
 0.500000    2323
 0.800000     925
 0.600000     646
 0.700000     594
             ... 
-0.053788       1
 0.642500       1
 0.359184       1
 0.187216       1
-0.012500       1
Name: sentiment, Length: 1912, dtype: int64
3    10656
4     6536
2      767
5      516
1       68
Name: Target_label, dtype: int64
[14:09:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Krishna\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


xgboost classcification model accuracy(in %): 100.0


In [4]:
import numpy as np
sample_Input=input("enter a sentence")
while sample_Input!='q':
 #sample_Input=input("enter a sentence")
 n_sample=normalizer(sample_Input)
 
 processed_string=' '.join(n_sample)
    
 score=sentiment(processed_string)
 #print(np.asarray([score]))
 ypd=clf2.predict(np.asarray([score]))
 print(ypd)
 sample_Input = input("enter a sentence,q to exit")


enter a sentenceq


In [5]:
import pickle


In [6]:
final_df.head(1)

,sentiment,Target_label
0,0.0,3


In [7]:
clf2.predict(np.asarray([0.0]))

array([3], dtype=int64)

In [8]:
with open('review.pkl','wb') as f:
    pickle.dump(clf2,f)

In [9]:
clf2_model=pickle.load(open('review.pkl','rb'))

In [10]:
clf2_model.predict(np.asarray([0.0]))

array([3], dtype=int64)

In [13]:
test_data.dtypes

date                  object
Review Text           object
category              object
Unnamed: 3            object
Unnamed: 4            object
                      ...   
Unnamed: 78           object
normalized_Tweets     object
processed_tweets      object
sentiment            float64
Target_label           int64
Length: 83, dtype: object

In [17]:
from sklearn.model_selection import RandomizedSearchCV
params = {'max_depth': np.arange(2, 10),
          'n_estimators': [5, 10, 15, 20, 25],
          'colsample_bytree': [0.3, 0.7],
          'subsample': [0.4, 0.6, 0.8, 1.0]
         }
randomized_cv = RandomizedSearchCV(estimator=clf2,param_distributions=params, scoring='roc_auc', n_iter=5, cv=5, verbose=1, n_jobs=1, 
                                   return_train_score=True)
randomized_cv.fit(x_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


C:\Users\Krishna\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:59:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 200, in __call__
    sample_weight=sample_weight)
  File "C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  UserWarning,
C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most rec

[14:59:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 200, in __call__
    sample_weight=sample_weight)
  File "C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  UserWarning,
C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most rec

[14:59:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 200, in __call__
    sample_weight=sample_weight)
  File "C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  UserWarning,
C:\Users\Krishna\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most rec

[14:59:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           gpu_id=-1, importance_type='gain',
                                           interaction_constraints='',
                                           learning_rate=0.300000012,
                                           max_delta_step=0, max_depth=6,
                                           min_child_weight=1, missing=nan,
                                           monotone_constraints='()',
                                           n_estimators=100, n_jobs=4,
                                           num_pa...
                                           random_state=0, reg_alpha=0,
                                           reg_lambda=1, scale_p

In [18]:
print("Best parameters found: ", randomized_cv.best_params_)

print("Best score found: ", randomized_cv.best_score_)

Best parameters found:  {'subsample': 1.0, 'n_estimators': 15, 'max_depth': 2, 'colsample_bytree': 0.7}
Best score found:  nan
